In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sms-spam-collection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F483%2F982%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240527%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240527T035342Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D44d83509c01f65605819d9db52cc9ae4678feade1d22a11a979ac490b3a2cd62765a545a2b08aa777817c51dc0724c36c035e674e7778cd4bca4d2cb6960ab7084f184cf97528e6ca05b63dcdbb5111a3a2c25b100acbd50851a1d418848657abc7957a3c9282991574526d4a54f6cc642415cec6c23f58e0595a59a44db814f84bfbc2f5b591321913098c0ba657457b42e14071d44516b24c6287f5babb78fbe20d34a85e91851940cfec0e04e7951973b1aaee1b2114ad3997bcfe0e6b7e325a25a5b88e4c2bb6684fdb9d646d457f7be7c24e6ea5f11b9d501b49dfb3ddc41be1f423f3f5152654e60f88342b835b2ed1b570fb6815c74c6ce4ff8e5e74a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='latin1')
data.head()


In [ ]:
data = data.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
data

In [ ]:
data = data.rename(columns = {'v1':'Category','v2':'sms'})
data

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['Category'] = encoder.fit_transform(data['Category'])

data

In [ ]:
data = data.drop_duplicates()
data.info()

In [ ]:
import matplotlib.pyplot as plt

data.Category.plot.hist()
data.Category.value_counts()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
#determining the number of characters
data['No_of_characters'] = data['sms'].apply(len)
data

In [ ]:
data['No_of_words'] = data['sms'].apply(lambda x:len(nltk.word_tokenize(x)))
data

In [ ]:
data['No_of_sentences'] = data['sms'].apply(lambda x:len(nltk.sent_tokenize(x)))
data

In [ ]:
data[data['Category']==1][['No_of_characters','No_of_words','No_of_sentences']].describe()

In [ ]:
import seaborn as sns
new = data.drop(columns = 'sms')
sns.heatmap(new.corr(),annot = True)

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import string
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    x = []
    for i in y:
        if i not in stopwords.words('english') and i not in string.punctuation:
            x.append(i)

    y = x[:]
    x =[]
    for i in y:
        x.append(ps.stem(i))

    return " ".join(x)

In [ ]:
data['Transformed'] = data['sms'].apply(transform_text)

In [ ]:
data

In [ ]:
data[['No_of_characters','No_of_words','No_of_sentences']].corrwith(data['Category']).plot.bar(figsize=(16,9), title = 'Correlation with the variables', rot =45, grid=True)

In [ ]:
spam_corpus = []
for elem in data['Transformed'][data['Category']==1].to_list():
    for word in elem.split():
        spam_corpus.append(word)

len(spam_corpus)

In [ ]:
from collections import Counter
import seaborn as sns
word = pd.DataFrame(Counter(spam_corpus).most_common(30))

sns.barplot(x = word[0], y = word[1],)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Create separate word clouds for spam and ham messages
spam_words = ' '.join(list(data[data['Category'] == 1]['Transformed']))
ham_words = ' '.join(list(data[data['Category'] == 0]['Transformed']))

# Generate word clouds
spam_wordcloud = WordCloud(width=600, height=400, background_color='black').generate(spam_words)
ham_wordcloud = WordCloud(width=600, height=400, background_color='black').generate(ham_words)

# Display word clouds
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(spam_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Spam Words')

plt.subplot(1, 2, 2)
plt.imshow(ham_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Ham Words')

plt.show()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Transformed']).toarray()
X.shape


In [ ]:
y = data['Category'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create different Naive Bayes models
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

# Fit the models with the training data
gnb.fit(X_train, y_train)
mnb.fit(X_train, y_train)
bnb.fit(X_train, y_train)

# Predict the labels for the test data
y_pred_gnb = gnb.predict(X_test)
y_pred_mnb = mnb.predict(X_test)
y_pred_bnb = bnb.predict(X_test)

# Calculate the accuracy scores for each model
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)

# Print the accuracy scores
print("Gaussian Naive Bayes accuracy:", accuracy_gnb)
print("Multinomial Naive Bayes accuracy:", accuracy_mnb)
print("Bernoulli Naive Bayes accuracy:", accuracy_bnb)

In [ ]:
from sklearn.metrics import precision_score

# Calculate the precision scores for each model
precision_gnb = precision_score(y_test, y_pred_gnb)
precision_mnb = precision_score(y_test, y_pred_mnb)
precision_bnb = precision_score(y_test, y_pred_bnb)

# Print the precision scores
print("Gaussian Naive Bayes precision:", precision_gnb)
print("Multinomial Naive Bayes precision:", precision_mnb)
print("Bernoulli Naive Bayes precision:", precision_bnb)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=3000)

# Fit and transform the transformed text data
X = vectorizer.fit_transform(data['Transformed'])

# Convert the sparse matrix to a dense array
X = X.toarray()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create different Naive Bayes models
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

# Fit the models with the training data
gnb.fit(X_train, y_train)
mnb.fit(X_train, y_train)
bnb.fit(X_train, y_train)

# Predict the labels for the test data
y_pred_gnb = gnb.predict(X_test)
y_pred_mnb = mnb.predict(X_test)
y_pred_bnb = bnb.predict(X_test)

# Calculate the accuracy scores for each model
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)

# Print the accuracy scores
print("Gaussian Naive Bayes accuracy:", accuracy_gnb)
print("Multinomial Naive Bayes accuracy:", accuracy_mnb)
print("Bernoulli Naive Bayes accuracy:", accuracy_bnb)


In [ ]:
# Calculate the precision scores for each model
precision_gnb = precision_score(y_test, y_pred_gnb)
precision_mnb = precision_score(y_test, y_pred_mnb)
precision_bnb = precision_score(y_test, y_pred_bnb)

# Print the precision scores
print("Gaussian Naive Bayes precision:", precision_gnb)
print("Multinomial Naive Bayes precision:", precision_mnb)
print("Bernoulli Naive Bayes precision:", precision_bnb)


In [ ]:
#maximum precision and accuracy is in Multinomial Native Bayes
import pickle
pickle.dump(vectorizer,open('Vectorizer.pkl','wb'))
pickle.dump(mnb, open('model.pkl','wb'))